In [ ]:
import matplotlib.pyplot as plt
from collections import namedtuple

from functools import reduce
# from funcy import compose

from collections import Counter
import numpy as np
import pandas as pd

from operator import add

from bogatron import draw_pdf_contours, Dirichlet, xy2bc, trimesh
# draw_pdf_contours(Dirichlet([200, 100, 100]))

In [ ]:
def js_to_python(s):
    d = {
        "true": "True",
        "false": "False",
        "null": "None",
        "[]": "None",
        "][": "],["
    }
    for k, v in d.items():
        s = s.replace(k, v)
    return s

def read_line(s):
    try:
        return eval(js_to_python(s))
    except Exception as e:
        print(e)
        a, b = eval(js_to_python(s))
        return a

def listdecorator(f):
    return lambda x: list(f(x))

@listdecorator
def get_trial_data(exp_dicts):
    return filter(is_trial, exp_dicts)
        
def is_trial(d):
    st = 'stimulus_type'
    return (d[st] == 'test') if (st in d.keys()) else False

g0_instruction = '\n<b>Geben Sie in jeder Runde Ihr Bestes, um zu gewinnen und beenden Sie die Studie bitte erst, wenn Sie alle Runden beendet haben.</b>\n'
g1_instruction = '\n<b>Geben Sie in jeder Runde Ihr Bestes, um zu gewinnen, indem Sie versuchen\n    möglichst zufällig zu spielen, da dies die beste Strategie ist, um eine\n    künstliche Intelligenz zu besiegen.</b>\n'

def get_group(vp):
    return int(raw_data[vp][2]['stimulus'] == g1_instruction)

def nonempty_vp_idx(Ns, thr=0):
    return map(lambda x: x[0], (filter(lambda x: x[1] > thr, enumerate(Ns))))

def get_responses(vp_data):
    return list(map(lambda d: (d['response'], d['bot_response']), filter(lambda d: 'bot_response' in d.keys(), vp_data)))

def result(t):
    return 0 if t[0] == t[1] else -1 if t[0]+t[1] in 'RPSR' else 1

def count(vp):
    return Counter(map(result, get_responses(raw_data[vp]))) 

def compute_winrate(vp):
    r = count(vp)
    return r[1] / (r[-1] + r[1])

def to_counts(c):
    return Counts(win=c[1], lose=c[-1], draw=c[0])

def addcounts(c, acc):
    r = Counts(win=acc.win + c.win, lose=acc.lose + c.lose, draw=acc.draw + c.draw)
    return r

def add_result(r):
    return r[0], r[1], result(r)

def plot_axis():
    plt.plot([0, 0.5], [0, 0.2877], 'k-')
    plt.plot([0.5, 0.5], [np.sqrt(0.75), 0.2877], 'k-')
    plt.plot([1, 0.5], [0.0, 0.2877], 'k-')
    
Counts = namedtuple('Counts', ['win', 'lose', 'draw'])

def to_counts(c):
    return Counts(win=c[1], lose=c[-1], draw=c[0])

def addcounts(c, acc):
    r = Counts(win=acc.win + c.win, lose=acc.lose + c.lose, draw=acc.draw + c.draw)
    return r


In [ ]:
# load data
fn = '../data/experiment/jatos_results_20230128043140.txt'
with open(fn) as fh:
    raw_data = [read_line(s) for s in fh]
    #raw_data = fh.readlines()
print('all vps:', len(raw_data))

vps = list(filter(lambda vp: len(raw_data[vp]) > 600, range(len(raw_data))))
print('vps with all trials:', vps)

group = {vp: get_group(vp) for vp in vps}
sum(group.values()) / len(group)

In [ ]:
# todo: pro gruppe statistik ueber post questionare antworten
# freies textfeld

# bot strategie:
# wie start ist die präferenz der VPs

In [ ]:
# TODO
# winstay loseshift
# randomness evaluierne
# todo: korreliert strategie mit randomness measure?
for vp in vps:
    pass
    #print(raw_data[vp][-1]['response'])

In [ ]:
#list(map(add_result, (reduce(add, [get_responses(raw_data[i]) for i in vps if group[i] == 0]))))
len(group), len(vps)

In [ ]:
# dataframe mit spalte result, gruppe, strategie
for c in range(2):
    pass
    pd.DataFrame(list(map(add_result, (reduce(add, [get_responses(raw_data[i]) for i in vps if group[i] == c]))))).to_excel(f'group{c}.xlsx')

In [ ]:
list(map(result, get_responses(raw_data[0])))
for i in vps:
    fn = f'excel_files/vp{i:02d}_group-{group[i]}.xlsx'
    pd.DataFrame(map(add_result, get_responses(raw_data[i]))).to_excel(fn)

In [ ]:
# strats = ['nash_equilibrium_strategy', 'winstay', 'rotate_strategy', 'learn_preference', 'dont_always_copy_opponent_move']
# strats = ['super_male_strategy', 'nash_equilibrium_strategy', 'winstay', 'rotate_strategy', 'learn_preference', 'dont_always_copy_opponent_move']
# for vp in vps:
#     strategy = strats[list(map(lambda d: d['strategy'], filter(lambda d: 'strategy' in d.keys(), raw_data[vp])))[0]]
#     print(f'VP {vp}: {compute_winrate(vp)}, Group {get_group(vp)}')

In [ ]:
# for vp in vps:
#     wr = compute_winrate(vp)
    
# sum([compute_winrate(vp) for vp in vps]) / len(vps)

In [ ]:
sum([compute_winrate(vp) for vp in vps if group[vp]]) / sum(group.values())

In [ ]:
sum([compute_winrate(vp) for vp in vps if not group[vp]]) / (len(group) - sum(group.values()))

In [ ]:
# Sum up all results
c_total = reduce(addcounts, map(to_counts, map(count, vps)), Counts(0, 0, 0))

In [ ]:
# reduce(addcounts, to_counts(count(0)), Counts(0, 0, 0))
vp = np.random.choice(vps)
c = to_counts(count(vp))
c

In [ ]:
fig, ax = plt.subplots(1, 2)
ax[0].set_title(f'VP{vp}: {c.win}, {c.lose}, {c.draw}')
plt.sca(ax[0])
draw_pdf_contours(Dirichlet((c.win, c.lose, c.draw)))
plot_axis()
plt.sca(ax[1])
ax[1].set_title(f'total: {c_total.win}, {c_total.lose}, {c_total.draw}')
draw_pdf_contours(Dirichlet((c_total.win, c_total.lose, c_total.draw)))
plot_axis()
plt.show()

In [ ]:
c = reduce(addcounts, map(to_counts, map(count, vps)), Counts(0, 0, 0))
draw_pdf_contours(Dirichlet((c.win, c.lose, c.draw)))

In [ ]:
fig, ax = plt.subplots(1, 2)
for i in range(2):
    c = reduce(addcounts, map(to_counts, map(count, filter(lambda vp: get_group(vp)==i, vps))), Counts(0, 0, 0))
    plt.sca(ax[i])
    draw_pdf_contours(Dirichlet((c.win, c.lose, c.draw)))

In [ ]:
for i in range(2):
    c = reduce(addcounts, map(to_counts, map(count, filter(lambda vp: get_group(vp)==i, vps))), Counts(0, 0, 0))
    print(c)

In [ ]:
for i in range(2):
    c = reduce(addcounts, map(to_counts, map(count, filter(lambda vp: get_group(vp)==i, vps))), Counts(0, 0, 0))
    draw_pdf_contours(Dirichlet((c.win, c.lose, c.draw)))
    plot_axis()
    plt.savefig(f'dirichlet_group{i}.png')

In [ ]:
# Hypothese 1: win stay lose shift

In [ ]:
def compress(uncompressed):
    """Compress a string to a list of output symbols."""

    # Build the dictionary.
    dict_size = 256
    dictionary = {chr(i): chr(i) for i in range(dict_size)}
    #dictionary = {a: a for a in "RPS"}

    w = ""
    result = []
    for c in uncompressed:
        wc = w + c
        if wc in dictionary:
            w = wc
            #print('continue', w)
        else:
            result.append(dictionary[w])
            # Add wc to the dictionary.
            dictionary[wc] = dict_size
            dict_size += 1
            #print('append', wc)
            w = c

    #print(w, dictionary)
    # Output the code for w.
    if w:
        result.append(dictionary[w])
    return result

In [ ]:
# How to use:
s = 'TOBEORNOTTOBEORTOBEORNOT'
compressed = compress(s)
compressed

In [ ]:
def decompress(compressed):
    """Decompress a list of output ks to a string."""
    from io import StringIO

    # Build the dictionary.
    dict_size = 256
    dictionary = {chr(i): chr(i) for i in range(dict_size)}

    # use StringIO, otherwise this becomes O(N^2)
    # due to string concatenation in a loop
    result = StringIO()
    w = compressed.pop(0)
    result.write(w)
    for k in compressed:
        if k in dictionary:
            entry = dictionary[k]
        elif k == dict_size:
            entry = w + w[0]
        else:
            raise ValueError('Bad compressed k: %s' % k)
        result.write(entry)

        # Add w+entry[0] to the dictionary.
        dictionary[dict_size] = w + entry[0]
        dict_size += 1

        w = entry
    return result.getvalue()


print (compressed)
decompressed = decompress(compressed)
print (decompressed)

In [ ]:
len(compressed)
dict_size = 256
dictionary = {chr(i): chr(i) for i in range(dict_size)}
# dictionary

In [ ]:
first = lambda x: x[0]


for vp in vps:
    responses = ''.join(map(first, get_responses(raw_data[vp])))
    compressed = compress(responses)
    print(vp, len(compressed) / 77.3)

In [ ]:
random_lzscores = [len(compress(''.join(map(lambda x: 'RPS'[x], np.random.randint(0, 3, size=200))))) for _ in range(10000)]
np.array(random_lzscores).mean()